In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
host = 'menace-10alytics.032716360714.us-east-1.redshift-serverless.amazonaws.com'
db = 'c5'
port = 5439 
username = 'menace'
password = 'Menace97'

In [3]:
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{db}')
engine.connect()

In [27]:
query = ''' SELECT * FROM logs_data LIMIT 4 '''

pd.read_sql(query,engine)

,date,user_id,event_name,mobile_os_type,version_type
0,1/1/2020,0741d2813527501f6c9fd2c66bd59c0e,push_received,U683CL,11.3.1
1,1/2/2020,f5d1bd9f7a7be974367019665910989e,push_received,U683CL,11.3.1
2,1/3/2020,325b61cf1c4dc2a51eaf7ded19a48d98,push_received,U683CL,11.3.0
3,1/4/2020,ad32a2073e48423c2cd7ff3da30065d8,screen_view,U683CL,11.3.1


In [14]:
import boto3

In [20]:
# Sales data download
# Create an S3 client
s3 = boto3.client('s3')

# Specify the bucket name and object key
bucket_name = 'tenalytics-c9'
object_key = 'data/sales_data.csv'

# Download the object to a local file
s3.download_file(bucket_name, object_key, 'sales_data.csv')


In [15]:
pwd()

'C:\\Users\\Temporary\\Documents'

In [21]:
# Customer data download
# Create an S3 client
s3 = boto3.client('s3')

# Specify the bucket name and object key
bucket_name = 'tenalytics-c9'
object_key = 'data/customer_profiles.csv'

# Download the object to a local file
s3.download_file(bucket_name, object_key, 'customer_profiles.csv')


# Redshift console

In [4]:
# The data was loaded into redshift using the web console
# Now is time to for exploratory data analysis 
query = ''' SELECT * FROM sales_data ORDER BY order_id  LIMIT 4 '''

pd.read_sql(query,engine)

,order_id,product_name,product_category,price,quantity,sale_date,customer_id
0,1,life,Hydration,70.39,9,2022-04-28,4856
1,2,worry,Electronics,22.60,10,2022-06-29,5142
2,3,city,Hydration,23.54,7,2022-09-23,3710
3,4,form,Hydration,15.21,8,2022-08-14,3817


In [5]:
query = ''' SELECT * FROM customer_profiles ORDER BY customer_id  LIMIT 4 '''

pd.read_sql(query,engine)

,customer_id,first_name,last_name,gender,location,join_date,customer_segment
0,1,Stephanie,Montoya,Female,Tennessee,2021-06-23,Organic Advocate
1,2,Howard,Hall,Female,North Carolina,2022-06-06,Waste Reducer
2,3,Jennifer,Gardner,Male,Massachusetts,2022-03-04,Recycler
3,4,Gabrielle,Flores,Female,Missouri,2022-09-25,Waste Reducer


Calculate the total sales revenue generated from sustainable 
products each month for the year 2022. Sustainable products are 
those that meet specific criteria for eco-friendliness.


In [20]:
query = ''' 
SELECT
    TO_CHAR(SALE_DATE::DATE, 'YYYY-MM') AS month,
    SUM(PRICE * QUANTITY) AS total_sales_revenue
FROM
    SALES_DATA
WHERE
    EXTRACT(YEAR FROM SALE_DATE::DATE) = 2022
    AND product_category = 'Hydration'
GROUP BY
    month
ORDER BY
    month;

'''

pd.read_sql(query,engine)

,month,total_sales_revenue
0,2022-01,39433.39
1,2022-02,47418.05
2,2022-03,48862.27
3,2022-04,49377.16
4,2022-05,48739.12
5,2022-06,52300.91
6,2022-07,46067.10
7,2022-08,42234.54
8,2022-09,49626.42
9,2022-10,43456.81


Identify the top five customer segments (based on customer 
profiles) that contribute the most to EcoMarket's revenue in 2022

In [14]:
query = ''' 
SELECT total_customer_segment 
FROM (SELECT
    cp.customer_segment AS total_customer_segment,
    SUM(sd.price * sd.quantity) 
FROM
    customer_profiles cp
JOIN
    sales_data sd
    ON cp.customer_id = sd.customer_id
WHERE
    EXTRACT(YEAR FROM TO_DATE(sd.sale_date, 'YYYY-MM-DD')) = 2022
GROUP BY
    cp.customer_segment
ORDER BY
    total_customer_segment DESC
LIMIT 5)

'''

pd.read_sql(query,engine)

,total_customer_segment
0,427034.19
1,419151.91
2,417520.78
3,411112.01
4,410613.30


In [25]:
# This is the sql code for my report
query = ''' 
SELECT * 
FROM
(SELECT
    TO_CHAR(SALE_DATE::DATE, 'YYYY-MM') AS month,
    SUM(PRICE * QUANTITY) AS total_sales_revenue
FROM
    SALES_DATA
WHERE
    EXTRACT(YEAR FROM SALE_DATE::DATE) = 2022
    AND product_category = 'Hydration'
GROUP BY
    month
ORDER BY
    month) as a,
(SELECT total_customer_segment 
FROM (SELECT
    cp.customer_segment AS total_customer_segment,
    SUM(sd.price * sd.quantity)
FROM
    customer_profiles cp
JOIN
    sales_data sd
    ON cp.customer_id = sd.customer_id
WHERE
    EXTRACT(YEAR FROM TO_DATE(sd.sale_date, 'YYYY-MM-DD')) = 2022
GROUP BY
    cp.customer_segment
ORDER BY
    total_customer_segment DESC
LIMIT 5) as total_customer_segment) as b

order by total_sales_revenue DESC
'''

df = pd.read_sql(query,engine)

In [26]:
df.to_sql('eco_sales_insights',if_exists='replace',con=engine,index=False)

60

In [ ]:
df.to_DatFrame

In [ ]:
# Sales data download
# Create an S3 client
s3 = boto3.client('s3')

# Specify the bucket name and object key
bucket_name = 'tenalytics-c9'
object_key = 'data/sales_data.csv'

# Download the object to a local file
s3.download_file(bucket_name, object_key, 'sales_data.csv')
